<a href="https://colab.research.google.com/github/navneethosmane1499/Googled-Lens/blob/master/Image_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! kaggle competitions download -c aptos2019-blindness-detection

100% 9.49G/9.51G [01:31<00:00, 99.7MB/s]
100% 9.51G/9.51G [01:31<00:00, 111MB/s] 


In [ ]:
! unzip aptos2019-blindness-detection.zip

In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
file_name = []
file_path = []
id_code = []
for image in os.listdir('/content/drive/MyDrive/processed_test_images'):
  file_name.append(image)
  file_path.append('/content/drive/MyDrive/processed_test_images/'+image)
  id_code.append(image.replace('.png', ''))
test_df = pd.DataFrame(list(zip(id_code, file_path, file_name)), columns=['id_code', 'file_path', 'file_name'])
test_df.to_csv('/content/drive/MyDrive/test_df.csv')

In [ ]:
file_name = []
file_path = []
id_code = []
for image in os.listdir('/content/drive/MyDrive/processed_trained_images'):
  file_name.append(image)
  file_path.append('/content/drive/MyDrive/processed_trained_images/'+image)
  id_code.append(image.replace('.png', ''))
train_labels_df = pd.read_csv('/content/train.csv')
train_df = pd.merge(pd.DataFrame(list(zip(id_code, file_path, file_name)), columns=['id_code', 'file_path', 'file_name']), train_labels_df, on='id_code')
train_df.to_csv('/content/drive/MyDrive/train_df.csv')

In [ ]:
train_df = pd.read_csv('train_df.csv')
test_df = pd.read_csv('test_df.csv')
print(train_df.shape, test_df.shape)

In [ ]:
train_df['diagnosis'] = train_df['diagnosis'].apply(str)
train_df.dtypes

Unnamed: 0     int64
id_code       object
file_path     object
file_name     object
diagnosis     object
dtype: object

In [ ]:
IMG_SIZE = 512
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

In [ ]:
def circle_crop(img, sigmaX = 30):   
    """
    Create circular crop around image centre    
    """    
    img = crop_image_from_gray(img)    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    height, width, depth = img.shape    
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    img=cv2.addWeighted(img,4, cv2.GaussianBlur( img , (0,0) , sigmaX) ,-4 ,128)
    return img 

In [ ]:
for image in os.listdir('/content/train_images'):
    img = cv2.imread('/content/train_images/'+image)
    img = circle_crop(img) 
    cv2.imwrite('/content/drive/MyDrive/processed_trained_images/'+image, cv2.resize(img, (IMG_SIZE,IMG_SIZE)))

In [ ]:
for image in os.listdir('/content/test_images'):
    img = cv2.imread('/content/test_images/'+image)
    img = circle_crop(img) 
    cv2.imwrite('/content/drive/MyDrive/processed_test_images/'+image, cv2.resize(img, (IMG_SIZE,IMG_SIZE)))